# European Develompent Indicators - Iceland

## Research Questions

1. How has Iceland's GDP growth correlated with inflation rates over the past 20 years?

2. What is the relationship between educational attainment and unemployment in Iceland?

3. What is the gender wage gap trend in Iceland, and how does it compare with female employment rates?

## Datasets

* [namq_10_gdp](https://doi.org/10.2908/NAMQ_10_GDP) - Eurostat: Quarterly GDP

* [nama_10_co3_p3](https://doi.org/10.2908/NAMA_10_CO3_P3) - Eurostat: Annual household disposable income

* [tec00001](https://doi.org/10.2908/TEC00001) - Eurostat: Annual GDP at market prices

* [prc_hicp_midx](https://doi.org/10.2908/PRC_HICP_MIDX) - Eurostat: Monthly Harmonized CPI

* [prc_hicp_manr](https://doi.org/10.2908/PRC_HICP_MANR) - Eurostat: Monthly CPI inflation rate

* [prc_hicp_inw](https://doi.org/10.2908/PRC_HICP_INW) - Eurostat: Annual CPI inflation rate by income group

* [lfsa_urgaed](https://doi.org/10.2908/LFSA_URGAED) - Eurostat: Annual unemployment rate by education level

* [lfsa_egaed](https://doi.org/10.2908/LFSA_EGAED) - Eurostat: Annual employment rate by education level

* [lfsa_egised](https://doi.org/10.2908/LFSA_EGISED) - Eurostat: Annual employed persons by occupation and educational attainment level

* [lfsa_egan](https://doi.org/10.2908/LFSA_EGAN) - Eurostat: Annual employment

* [lfsi_emp_q](https://doi.org/10.2908/LFSI_EMP_Q) - Eurostat: Quarterly employment and activity by sex and age

* [lfsi_long_q](https://doi.org/10.2908/LFSI_LONG_Q) - Eurostat: Quarterly labour market transitions

* [lfsq_ergan](https://doi.org/10.2908/LFSQ_ERGAN) - Eurostat: Quarterly employment rates by citizenship

* [une_rt_m](https://doi.org/10.2908/UNE_RT_M) - Eurostat: Monthly total unemployment rate

* [edat_lfse_03](https://doi.org/10.2908/EDAT_LFSE_03) - Eurostat: Annual population by educational attainment level

* [earn_nt_net](https://doi.org/10.2908/EARN_NT_NET) - Eurostat: Annual net earnings

* [earn_gr_gpgr2](https://doi.org/10.2908/EARN_GR_GPGR2) - Eurostat: Annual gender pay gap in unadjusted form


## Imports

In [1]:
import os
import eurostat
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from src.frame_class import Frame, Frequency, Category

## Parameters

In [2]:
COUNTRY = {
    "name": "Iceland",
    "code": "IS"
}
DATAFRAMES = [
    Frame(
        name="Quarterly GDP",
        file_name="quarterly_gdp.csv",
        description="Gross domestic product (GDP) and main components (output, expenditure and income)",
        eurostat_code="namq_10_gdp",
        frequency=Frequency.QUARTERLY,
        useful_column_filter = {
            "na_item": ["B1GQ"],
            "unit": ["CLV_PCH_SM", "CLV_PCH_PRE"],  # fallback included
            "s_adj": ["SCA", "SA", "NSA"]          # preference order
        },
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Annual Household Disposable Income",
        file_name="annual_household_disposable_income.csv",
        description="Household final consumption expenditure by purpose (COICOP 1999)",
        eurostat_code="nama_10_co3_p3",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "coicop": ["TOTAL"],
            "unit": ["CP_EUR_HAB"]
        },
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Annual GDP at Market Prices",
        file_name="annual_gdp_at_market_prices.csv",
        description="Gross domestic product at market prices",
        eurostat_code="tec00001",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "na_item": ["B1GQ"],
            "unit": ["CP_EUR_HAB"]
        },
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Monthly Harmonized CPI",
        file_name="monthly_harmonized_cpi.csv",
        description="HICP - monthly data (index)",
        eurostat_code="prc_hicp_midx",
        frequency=Frequency.MONTHLY,
        useful_column_filter = {
            "unit": ["I15"],
            "coicop": ["TOT_X_NRG_FOOD", "TOT_X_ALC_TBC"]
        },
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Monthly CPI Inflation Rate",
        file_name="monthly_cpi_inflation_rate.csv",
        description="HICP - monthly data (annual rate of change)",
        eurostat_code="prc_hicp_manr",
        frequency=Frequency.MONTHLY,
        useful_column_filter = {
            "unit": ["RCH_A"],
            "coicop": ["TOT_X_NRG_FOOD", "TOT_X_TBC"]  # core + headline
        },
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Annual CPI Inflation Rate by Income Group",
        file_name="annual_cpi_inflation_rate_by_income_group.csv",
        description="HICP - item weights",
        eurostat_code="prc_hicp_inw",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "unit": ["rate"],
            "coicop": ["TOT_X_NRG_FOOD", "TOT_X_TBC"]
        },
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Annual Unemployment Rate by Education Level",
        file_name="annual_unemployment_rate_by_education_level.csv",
        description="Unemployment rates by educational attainment level",
        eurostat_code="lfsa_urgaed",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "unit": ["PC"],
            "sex": ["T"],
            "age": ["Y25-64"],
            "isced11": ["ED0-2", "ED3_4", "ED5-8"]  # low, medium, high education
        },
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employment Rate by Education Level",
        file_name="annual_employment_rate_by_education_level.csv",
        description="Employed persons by educational attainment level",
        eurostat_code="lfsa_egaed",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "unit": ["THS_PER"],  # persons employed
            "sex": ["T"],
            "age": ["Y25-64"],
            "isced11": ["TOTAL", "ED0-2", "ED3_4", "ED5-8"]
        },
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employed Persons by Occupation and Educational Attainment Level",
        file_name="annual_employed_persons_by_occupation_and_educational_attainment_level.csv",
        description="Employed persons by occupation and educational attainment level",
        eurostat_code="lfsa_egised",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employment",
        file_name="annual_employment.csv",
        description="Employed persons by citizenship",
        eurostat_code="lfsa_egan",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "unit": ["THS_PER"],
            "sex": ["T"],
            "age": ["Y25-64"],
            "citizen": ["TOTAL"]
        },
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly employment and activity by sex and age",
        file_name="quarterly_employment_and_activity_by_sex_and_age.csv",
        description="Employment and activity by sex and age - quarterly data",
        eurostat_code="lfsi_emp_q",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly labour market transitions",
        file_name="quarterly_labour_market_transitions.csv",
        description="Labour market transitions - quarterly data",
        eurostat_code="lfsi_long_q",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly employment rates by citizenship",
        file_name="quarterly_employment_rates_by_citizenship.csv",
        description="Employment rates by citizenship - quarterly data",
        eurostat_code="lfsq_ergan",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Monthly total unemployment rate",
        file_name="monthly_total_unemployment_rate.csv",
        description="Unemployment by sex and age - monthly data",
        eurostat_code="une_rt_m",
        frequency=Frequency.MONTHLY,
        useful_column_filter = {
            "s_adj": ["SA"],
            "age": ["TOTAL"],
            "sex": ["T"],
            "unit": ["PC_ACT"]
        },
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Annual population by educational attainment level",
        file_name="annual_population_by_educational_attainment_level.csv",
        description="Population in private households by educational attainment level - main indicators",
        eurostat_code="edat_lfse_03",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "sex": ["T"],
            "age": ["Y25-64"],
            "unit": ["PC"],
            "isced11": ["ED0-2", "ED3_4", "ED5-8"]
        },
        categories=[Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual net earnings",
        file_name="annual_net_earnings.csv",
        description="Annual net earnings",
        eurostat_code="earn_nt_net",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "unit": ["EUR"],
            "estruct": ["NET"],
            "ecase": ["P1_NCH_AW100"]  # single, average wage, no children
        },
        categories=[Category.EARNINGS_AND_INEQUALITY]
    ),
    Frame(
        name="Annual gender pay gap in unadjusted form",
        file_name="annual_gender_pay_gap_in_unadjusted_form.csv",
        description="Gender pay gap in unadjusted form by NACE Rev. 2 activity - structure of earnings survey methodology",
        eurostat_code="earn_gr_gpgr2",
        frequency=Frequency.ANNUAL,
        useful_column_filter = {
            "unit": ["PC"],
            "nace_r2": ["B-S"]  # whole economy
        },
        categories=[Category.EARNINGS_AND_INEQUALITY]
    )
]

DATA_PATH = "data/"

print(f"Frame count: {len(DATAFRAMES)}")

Frame count: 17


In [3]:
for frame in DATAFRAMES:
    df: pd.DataFrame | None = None

    if os.path.exists(os.path.join("original_data", frame.file_name)):
        df = pd.read_csv(os.path.join("original_data", frame.file_name))
        print(f"Loaded {frame.name} dataset from local CSV.")

    else:
        try:
            df = eurostat.get_data_df(frame.eurostat_code, flags=False)
            os.makedirs("original_data", exist_ok=True)
            df.to_csv(os.path.join("original_data", frame.file_name), index=False)

            print(f"Downloaded and saved {frame.name} dataset.")

        except Exception as e:
            print(f"Failed to download {frame.name}: {e}")

    df.rename(columns={"geo\\TIME_PERIOD": "geo"}, inplace=True)
    df = df[df["geo"] == COUNTRY["code"]]
    df = df.dropna(axis=1, how='all')

    if "currency" in df.columns:
        df.rename(columns={"currency": "unit"}, inplace=True)

    if "freq" in df.columns:
        df = df.drop(columns=["freq"])

    if "unit" not in df.columns:
        df.insert(0, "unit", "rate")

    if frame.useful_column_filter:
        for col, valid_values in frame.useful_column_filter.items():
            if col in df.columns:
                df = df[df[col].isin(valid_values)]

    frame.dataframe = df

    os.makedirs(DATA_PATH, exist_ok=True)
    frame.dataframe.to_csv(os.path.join(DATA_PATH, frame.file_name), index=False)


Loaded Quarterly GDP dataset from local CSV.
Loaded Annual Household Disposable Income dataset from local CSV.
Loaded Annual GDP at Market Prices dataset from local CSV.
Loaded Monthly Harmonized CPI dataset from local CSV.
Loaded Monthly CPI Inflation Rate dataset from local CSV.
Loaded Annual CPI Inflation Rate by Income Group dataset from local CSV.
Loaded Annual Unemployment Rate by Education Level dataset from local CSV.
Loaded Annual Employment Rate by Education Level dataset from local CSV.
Loaded Annual Employed Persons by Occupation and Educational Attainment Level dataset from local CSV.
Loaded Annual Employment dataset from local CSV.
Loaded Quarterly employment and activity by sex and age dataset from local CSV.
Loaded Quarterly labour market transitions dataset from local CSV.
Loaded Quarterly employment rates by citizenship dataset from local CSV.
Loaded Monthly total unemployment rate dataset from local CSV.
Loaded Annual population by educational attainment level datase

In [4]:
for frame in DATAFRAMES:
    column_names = frame.dataframe.columns.tolist()
    time_period_index = column_names.index("geo")
    data_columns = column_names[time_period_index + 1 :]

    if frame.frequency == Frequency.MONTHLY:
        frame.dataframe.rename(columns={col: f"{col[:4]}-{col[5:7]}-01T00:00:00" for col in data_columns}, inplace=True)
    elif frame.frequency == Frequency.QUARTERLY:
        frame.dataframe.rename(columns={col: f"{col[:4]}-{(int(col[6]) - 1) * 3 + 1:02d}-01T00:00:00" for col in data_columns}, inplace=True)
    elif frame.frequency == Frequency.ANNUAL:
        frame.dataframe.rename(columns={col: f"{col}-01-01T00:00:00" for col in data_columns}, inplace=True)

    display(frame.dataframe.head(10))

,unit,s_adj,na_item,geo,1995-01-01T00:00:00,1995-04-01T00:00:00,1995-07-01T00:00:00,1995-10-01T00:00:00,1996-01-01T00:00:00,1996-04-01T00:00:00,...,2023-01-01T00:00:00,2023-04-01T00:00:00,2023-07-01T00:00:00,2023-10-01T00:00:00,2024-01-01T00:00:00,2024-04-01T00:00:00,2024-07-01T00:00:00,2024-10-01T00:00:00,2025-01-01T00:00:00,2025-04-01T00:00:00
25695,CLV_PCH_PRE,SCA,B1GQ,IS,NaN,2.0,0.9,-0.7,3.5,0.2,...,0.0,2.5,-3.4,0.6,-1.0,2.0,-0.7,-2.2,2.9,-0.7
26776,CLV_PCH_SM,NSA,B1GQ,IS,NaN,NaN,NaN,NaN,8.4,4.3,...,10.2,8.1,3.7,-0.4,-2.2,0.2,-0.7,-1.2,2.7,-1.9
27814,CLV_PCH_SM,SCA,B1GQ,IS,NaN,NaN,NaN,NaN,5.9,4.0,...,9.5,8.8,3.3,-0.4,-1.3,-1.8,1.0,-1.8,2.1,-0.7


,unit,coicop,geo,1995-01-01T00:00:00,1996-01-01T00:00:00,1997-01-01T00:00:00,1998-01-01T00:00:00,1999-01-01T00:00:00,2000-01-01T00:00:00,2001-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
29372,CP_EUR_HAB,TOTAL,IS,11030.0,11820.0,13200.0,14740.0,16670.0,19160.0,16650.0,...,24760.0,29790.0,34440.0,33970.0,32480.0,26690.0,30950.0,37820.0,39250.0,40210.0


,na_item,unit,geo,2013-01-01T00:00:00,2014-01-01T00:00:00,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
20,B1GQ,CP_EUR_HAB,IS,38460.0,42350.0,49180.0,57900.0,65850.0,65330.0,63430.0,54190.0,61360.0,74950.0,77210.0,79480.0


,unit,coicop,geo,1996-01-01T00:00:00,1996-02-01T00:00:00,1996-03-01T00:00:00,1996-04-01T00:00:00,1996-05-01T00:00:00,1996-06-01T00:00:00,1996-07-01T00:00:00,...,2024-12-01T00:00:00,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00
24535,I15,TOT_X_ALC_TBC,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.91,127.70,129.05,129.66,130.65,131.28,133.75,134.42,134.18,133.64
24926,I15,TOT_X_NRG_FOOD,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123.37,122.41,123.73,124.35,125.43,126.14,129.16,129.94,129.69,128.97


,unit,coicop,geo,1997-01-01T00:00:00,1997-02-01T00:00:00,1997-03-01T00:00:00,1997-04-01T00:00:00,1997-05-01T00:00:00,1997-06-01T00:00:00,1997-07-01T00:00:00,...,2024-12-01T00:00:00,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00
17565,RCH_A,TOT_X_NRG_FOOD,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.7,3.6,3.0,2.9,3.5,3.0,3.6,3.4,3.4,4.0
17697,RCH_A,TOT_X_TBC,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.6,3.7,3.3,3.3,3.8,3.4,3.8,3.6,3.6,4.2


...

,unit,sex,age,citizen,geo,1998-04-01T00:00:00,1999-04-01T00:00:00,2000-04-01T00:00:00,2001-04-01T00:00:00,2002-04-01T00:00:00,...,2023-01-01T00:00:00,2023-04-01T00:00:00,2023-07-01T00:00:00,2023-10-01T00:00:00,2024-01-01T00:00:00,2024-04-01T00:00:00,2024-07-01T00:00:00,2024-10-01T00:00:00,2025-01-01T00:00:00,2025-04-01T00:00:00
19,PC,F,Y15-19,EU27_2020_FOR,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,PC,F,Y15-19,FOR,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,PC,F,Y15-19,NAT,IS,61.8,66.7,76.6,73.0,61.7,...,67.9,73.3,80.5,66.4,73.3,68.3,75.0,60.3,64.0,68.3
130,PC,F,Y15-19,NEU27_2020_FOR,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,PC,F,Y15-19,NRP,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,PC,F,Y15-19,TOTAL,IS,61.4,67.3,76.9,73.4,62.5,...,67.9,73.3,76.7,66.4,72.8,69.1,73.2,60.2,64.0,68.6
262,PC,F,Y15-24,EU27_2020_FOR,IS,NaN,NaN,NaN,NaN,NaN,...,76.6,NaN,NaN,NaN,67.0,78.8,NaN,NaN,NaN,NaN
299,PC,F,Y15-24,FOR,IS,NaN,NaN,NaN,NaN,NaN,...,76.6,70.4,69.8,NaN,69.8,82.2,NaN,91.0,NaN,NaN
336,PC,F,Y15-24,NAT,IS,68.4,71.0,77.5,75.0,66.9,...,71.2,75.2,82.8,73.9,77.3,77.9,78.4,68.0,74.1,82.7
373,PC,F,Y15-24,NEU27_2020_FOR,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,s_adj,age,unit,sex,geo,2003-01-01T00:00:00,2003-02-01T00:00:00,2003-03-01T00:00:00,2003-04-01T00:00:00,2003-05-01T00:00:00,...,2024-12-01T00:00:00,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00
759,SA,TOTAL,PC_ACT,T,IS,3.8,3.7,3.6,3.6,3.5,...,3.6,3.7,3.7,3.7,3.7,3.7,3.8,3.8,3.9,3.9


,sex,age,unit,isced11,geo,1999-01-01T00:00:00,2000-01-01T00:00:00,2001-01-01T00:00:00,2002-01-01T00:00:00,2003-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
5173,T,Y25-64,PC,ED0-2,IS,43.3,44.2,43.7,41.3,40.4,...,25.0,21.8,22.7,22.0,21.4,21.8,18.0,18.7,18.0,17.5
5249,T,Y25-64,PC,ED3_4,IS,34.9,32.0,32.6,33.1,30.8,...,36.0,37.7,34.8,34.2,33.5,34.5,39.4,38.3,37.9,36.2
5357,T,Y25-64,PC,ED5-8,IS,21.8,23.8,23.8,25.6,28.8,...,39.0,40.5,42.5,43.8,45.2,43.7,42.6,42.9,44.0,46.3


,unit,estruct,ecase,geo,2000-01-01T00:00:00,2001-01-01T00:00:00,2002-01-01T00:00:00,2003-01-01T00:00:00,2004-01-01T00:00:00,2005-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
1302,EUR,NET,P1_NCH_AW100,IS,27860.32,25313.39,26541.59,28076.81,29732.44,38019.05,...,37027.37,44042.96,50456.07,48794.28,47254.37,44185.83,47374.82,53001.86,54539.33,57573.09


,unit,nace_r2,geo,2007-01-01T00:00:00,2008-01-01T00:00:00,2009-01-01T00:00:00,2010-01-01T00:00:00,2011-01-01T00:00:00,2012-01-01T00:00:00,2013-01-01T00:00:00,2014-01-01T00:00:00,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00
85,PC,B-S,IS,23.4,20.5,18.0,17.5,17.5,17.2,18.5,16.4,17.0,15.6,15.0,13.6,13.8,11.7,9.9,8.7,9.3


In [5]:
for frame in DATAFRAMES:
    column_names = frame.dataframe.columns.tolist()
    time_period_index = column_names.index("geo")
    data_columns = column_names[time_period_index + 1 :]
    id_cols = column_names[: time_period_index + 1]

    df_melted = frame.dataframe.melt(id_vars=id_cols, value_vars=data_columns, var_name="TIME_PERIOD", value_name="value")
    df_melted = df_melted.drop(columns=["geo"])
    frame.dataframe = df_melted
    display(frame.dataframe.head(10))

    os.makedirs("melted_data", exist_ok=True)
    frame.dataframe.to_csv(os.path.join("melted_data", frame.file_name), index=False)


,unit,s_adj,na_item,TIME_PERIOD,value
0,CLV_PCH_PRE,SCA,B1GQ,1995-01-01T00:00:00,NaN
1,CLV_PCH_SM,NSA,B1GQ,1995-01-01T00:00:00,NaN
2,CLV_PCH_SM,SCA,B1GQ,1995-01-01T00:00:00,NaN
3,CLV_PCH_PRE,SCA,B1GQ,1995-04-01T00:00:00,2.0
4,CLV_PCH_SM,NSA,B1GQ,1995-04-01T00:00:00,NaN
5,CLV_PCH_SM,SCA,B1GQ,1995-04-01T00:00:00,NaN
6,CLV_PCH_PRE,SCA,B1GQ,1995-07-01T00:00:00,0.9
7,CLV_PCH_SM,NSA,B1GQ,1995-07-01T00:00:00,NaN
8,CLV_PCH_SM,SCA,B1GQ,1995-07-01T00:00:00,NaN
9,CLV_PCH_PRE,SCA,B1GQ,1995-10-01T00:00:00,-0.7


,unit,coicop,TIME_PERIOD,value
0,CP_EUR_HAB,TOTAL,1995-01-01T00:00:00,11030.0
1,CP_EUR_HAB,TOTAL,1996-01-01T00:00:00,11820.0
2,CP_EUR_HAB,TOTAL,1997-01-01T00:00:00,13200.0
3,CP_EUR_HAB,TOTAL,1998-01-01T00:00:00,14740.0
4,CP_EUR_HAB,TOTAL,1999-01-01T00:00:00,16670.0
5,CP_EUR_HAB,TOTAL,2000-01-01T00:00:00,19160.0
6,CP_EUR_HAB,TOTAL,2001-01-01T00:00:00,16650.0
7,CP_EUR_HAB,TOTAL,2002-01-01T00:00:00,17570.0
8,CP_EUR_HAB,TOTAL,2003-01-01T00:00:00,18410.0
9,CP_EUR_HAB,TOTAL,2004-01-01T00:00:00,19950.0


,na_item,unit,TIME_PERIOD,value
0,B1GQ,CP_EUR_HAB,2013-01-01T00:00:00,38460.0
1,B1GQ,CP_EUR_HAB,2014-01-01T00:00:00,42350.0
2,B1GQ,CP_EUR_HAB,2015-01-01T00:00:00,49180.0
3,B1GQ,CP_EUR_HAB,2016-01-01T00:00:00,57900.0
4,B1GQ,CP_EUR_HAB,2017-01-01T00:00:00,65850.0
5,B1GQ,CP_EUR_HAB,2018-01-01T00:00:00,65330.0
6,B1GQ,CP_EUR_HAB,2019-01-01T00:00:00,63430.0
7,B1GQ,CP_EUR_HAB,2020-01-01T00:00:00,54190.0
8,B1GQ,CP_EUR_HAB,2021-01-01T00:00:00,61360.0
9,B1GQ,CP_EUR_HAB,2022-01-01T00:00:00,74950.0


,unit,coicop,TIME_PERIOD,value
0,I15,TOT_X_ALC_TBC,1996-01-01T00:00:00,NaN
1,I15,TOT_X_NRG_FOOD,1996-01-01T00:00:00,NaN
2,I15,TOT_X_ALC_TBC,1996-02-01T00:00:00,NaN
3,I15,TOT_X_NRG_FOOD,1996-02-01T00:00:00,NaN
4,I15,TOT_X_ALC_TBC,1996-03-01T00:00:00,NaN
5,I15,TOT_X_NRG_FOOD,1996-03-01T00:00:00,NaN
6,I15,TOT_X_ALC_TBC,1996-04-01T00:00:00,NaN
7,I15,TOT_X_NRG_FOOD,1996-04-01T00:00:00,NaN
8,I15,TOT_X_ALC_TBC,1996-05-01T00:00:00,NaN
9,I15,TOT_X_NRG_FOOD,1996-05-01T00:00:00,NaN


,unit,coicop,TIME_PERIOD,value
0,RCH_A,TOT_X_NRG_FOOD,1997-01-01T00:00:00,NaN
1,RCH_A,TOT_X_TBC,1997-01-01T00:00:00,NaN
2,RCH_A,TOT_X_NRG_FOOD,1997-02-01T00:00:00,NaN
3,RCH_A,TOT_X_TBC,1997-02-01T00:00:00,NaN
4,RCH_A,TOT_X_NRG_FOOD,1997-03-01T00:00:00,NaN
5,RCH_A,TOT_X_TBC,1997-03-01T00:00:00,NaN
6,RCH_A,TOT_X_NRG_FOOD,1997-04-01T00:00:00,NaN
7,RCH_A,TOT_X_TBC,1997-04-01T00:00:00,NaN
8,RCH_A,TOT_X_NRG_FOOD,1997-05-01T00:00:00,NaN
9,RCH_A,TOT_X_TBC,1997-05-01T00:00:00,NaN


...

,unit,sex,age,citizen,TIME_PERIOD,value
0,PC,F,Y15-19,EU27_2020_FOR,1998-04-01T00:00:00,NaN
1,PC,F,Y15-19,FOR,1998-04-01T00:00:00,NaN
2,PC,F,Y15-19,NAT,1998-04-01T00:00:00,61.8
3,PC,F,Y15-19,NEU27_2020_FOR,1998-04-01T00:00:00,NaN
4,PC,F,Y15-19,NRP,1998-04-01T00:00:00,NaN
5,PC,F,Y15-19,TOTAL,1998-04-01T00:00:00,61.4
6,PC,F,Y15-24,EU27_2020_FOR,1998-04-01T00:00:00,NaN
7,PC,F,Y15-24,FOR,1998-04-01T00:00:00,NaN
8,PC,F,Y15-24,NAT,1998-04-01T00:00:00,68.4
9,PC,F,Y15-24,NEU27_2020_FOR,1998-04-01T00:00:00,NaN


,s_adj,age,unit,sex,TIME_PERIOD,value
0,SA,TOTAL,PC_ACT,T,2003-01-01T00:00:00,3.8
1,SA,TOTAL,PC_ACT,T,2003-02-01T00:00:00,3.7
2,SA,TOTAL,PC_ACT,T,2003-03-01T00:00:00,3.6
3,SA,TOTAL,PC_ACT,T,2003-04-01T00:00:00,3.6
4,SA,TOTAL,PC_ACT,T,2003-05-01T00:00:00,3.5
5,SA,TOTAL,PC_ACT,T,2003-06-01T00:00:00,3.5
6,SA,TOTAL,PC_ACT,T,2003-07-01T00:00:00,3.5
7,SA,TOTAL,PC_ACT,T,2003-08-01T00:00:00,3.4
8,SA,TOTAL,PC_ACT,T,2003-09-01T00:00:00,3.4
9,SA,TOTAL,PC_ACT,T,2003-10-01T00:00:00,3.4


,sex,age,unit,isced11,TIME_PERIOD,value
0,T,Y25-64,PC,ED0-2,1999-01-01T00:00:00,43.3
1,T,Y25-64,PC,ED3_4,1999-01-01T00:00:00,34.9
2,T,Y25-64,PC,ED5-8,1999-01-01T00:00:00,21.8
3,T,Y25-64,PC,ED0-2,2000-01-01T00:00:00,44.2
4,T,Y25-64,PC,ED3_4,2000-01-01T00:00:00,32.0
5,T,Y25-64,PC,ED5-8,2000-01-01T00:00:00,23.8
6,T,Y25-64,PC,ED0-2,2001-01-01T00:00:00,43.7
7,T,Y25-64,PC,ED3_4,2001-01-01T00:00:00,32.6
8,T,Y25-64,PC,ED5-8,2001-01-01T00:00:00,23.8
9,T,Y25-64,PC,ED0-2,2002-01-01T00:00:00,41.3


,unit,estruct,ecase,TIME_PERIOD,value
0,EUR,NET,P1_NCH_AW100,2000-01-01T00:00:00,27860.32
1,EUR,NET,P1_NCH_AW100,2001-01-01T00:00:00,25313.39
2,EUR,NET,P1_NCH_AW100,2002-01-01T00:00:00,26541.59
3,EUR,NET,P1_NCH_AW100,2003-01-01T00:00:00,28076.81
4,EUR,NET,P1_NCH_AW100,2004-01-01T00:00:00,29732.44
5,EUR,NET,P1_NCH_AW100,2005-01-01T00:00:00,38019.05
6,EUR,NET,P1_NCH_AW100,2006-01-01T00:00:00,37806.34
7,EUR,NET,P1_NCH_AW100,2007-01-01T00:00:00,42383.61
8,EUR,NET,P1_NCH_AW100,2008-01-01T00:00:00,27556.77
9,EUR,NET,P1_NCH_AW100,2009-01-01T00:00:00,21683.70


,unit,nace_r2,TIME_PERIOD,value
0,PC,B-S,2007-01-01T00:00:00,23.4
1,PC,B-S,2008-01-01T00:00:00,20.5
2,PC,B-S,2009-01-01T00:00:00,18.0
3,PC,B-S,2010-01-01T00:00:00,17.5
4,PC,B-S,2011-01-01T00:00:00,17.5
5,PC,B-S,2012-01-01T00:00:00,17.2
6,PC,B-S,2013-01-01T00:00:00,18.5
7,PC,B-S,2014-01-01T00:00:00,16.4
8,PC,B-S,2015-01-01T00:00:00,17.0
9,PC,B-S,2016-01-01T00:00:00,15.6


In [6]:
for frame in DATAFRAMES:
    df = frame.dataframe.drop(columns=["TIME_PERIOD", "value"]).drop_duplicates()

    # if there is 'coicop' column, remove the values starting with CP
    if "coicop" in df.columns:
        df = df[~df["coicop"].astype(str).str.startswith("CP")]

    unique_values = {col: df[col].unique().tolist() for col in df.columns}
    print(f"Unique values for {frame.name}:")
    for col, values in unique_values.items():
        print(f"  {col}: {values}")
    print("\n")

Unique values for Quarterly GDP:
  unit: ['CLV_PCH_PRE', 'CLV_PCH_SM']
  s_adj: ['SCA', 'NSA']
  na_item: ['B1GQ']


Unique values for Annual Household Disposable Income:
  unit: ['CP_EUR_HAB']
  coicop: ['TOTAL']


Unique values for Annual GDP at Market Prices:
  na_item: ['B1GQ']
  unit: ['CP_EUR_HAB']


Unique values for Monthly Harmonized CPI:
  unit: ['I15']
  coicop: ['TOT_X_ALC_TBC', 'TOT_X_NRG_FOOD']


Unique values for Monthly CPI Inflation Rate:
  unit: ['RCH_A']
  coicop: ['TOT_X_NRG_FOOD', 'TOT_X_TBC']


Unique values for Annual CPI Inflation Rate by Income Group:
  unit: ['rate']
  coicop: ['TOT_X_NRG_FOOD', 'TOT_X_TBC']


Unique values for Annual Unemployment Rate by Education Level:
  unit: ['PC']
  sex: ['T']
  age: ['Y25-64']
  isced11: ['ED0-2', 'ED3_4', 'ED5-8']


Unique values for Annual Employment Rate by Education Level:
  unit: ['THS_PER']
  sex: ['T']
  age: ['Y25-64']
  isced11: ['ED0-2', 'ED3_4', 'ED5-8', 'TOTAL']


Unique values for Annual Employed Persons by

In [7]:
for frame in DATAFRAMES:
    # get all the unique column combinations except TIME_PERIOD and value
    unique_combinations = frame.dataframe.drop(columns=["TIME_PERIOD", "value"]).drop_duplicates()

    if "coicop" in unique_combinations.columns:
        unique_combinations = unique_combinations[~unique_combinations["coicop"].astype(str).str.startswith("CP")]

    # get the number of values for each combination
    unique_combinations["value_count"] = unique_combinations.apply(
        lambda row: frame.dataframe[
            (frame.dataframe.drop(columns=["TIME_PERIOD", "value"]) == row).all(axis=1)
        ].shape[0], axis=1
    )

    print(f"Unique column combinations for {frame.name}:")
    display(unique_combinations)

Unique column combinations for Quarterly GDP:


,unit,s_adj,na_item,value_count
0,CLV_PCH_PRE,SCA,B1GQ,122
1,CLV_PCH_SM,NSA,B1GQ,122
2,CLV_PCH_SM,SCA,B1GQ,122


Unique column combinations for Annual Household Disposable Income:


,unit,coicop,value_count
0,CP_EUR_HAB,TOTAL,30


Unique column combinations for Annual GDP at Market Prices:


...

,sex,age,unit,isced11,value_count
0,T,Y25-64,PC,ED0-2,26
1,T,Y25-64,PC,ED3_4,26
2,T,Y25-64,PC,ED5-8,26


Unique column combinations for Annual net earnings:


,unit,estruct,ecase,value_count
0,EUR,NET,P1_NCH_AW100,25


Unique column combinations for Annual gender pay gap in unadjusted form:


,unit,nace_r2,value_count
0,PC,B-S,17


In [8]:
for frame in DATAFRAMES:
    quantiles_25 = []
    quantiles_75 = []
    interquartile_ranges = []

    for unit in df["unit"].unique():
        unit_data = df[df["unit"] == unit]
        quantiles_25.append(unit_data["value"].quantile(0.25))
        quantiles_75.append(unit_data["value"].quantile(0.75))
        interquartile_ranges.append(quantiles_75[-1] - quantiles_25[-1])

    # quantile_25 = frame.dataframe["value"].quantile(0.25)
    # quantile_75 = frame.dataframe["value"].quantile(0.75)
    # interquartile_range = quantile_75 - quantile_25
    # outliers = frame.dataframe[(frame.dataframe["value"] < (quantile_25 - 1.5 * interquartile_range)) | (frame.dataframe["value"] > (quantile_75 + 1.5 * interquartile_range))]

    # display(outliers.head(10))

    # plt.figure(figsize=(10, 6))
    # sns.boxplot(x=frame.dataframe["value"])
    # plt.title(f"{frame.name} - Outlier Detection")
    # plt.show()


KeyError: 'value'